In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets sacrebleu rouge_score indic-nlp-library --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from tqdm.auto import tqdm

# Load the CSV
df = pd.read_csv('/content/drive/MyDrive/Dissertation/PMIndiaSum_test.csv')
df = df.dropna().reset_index(drop=True)
print(df.count())
df.head()

source_url    116
target_url    116
text          116
summary       116
dtype: int64


,source_url,target_url,text,summary
0,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്...,"ഇന്ത്യയും അംഗോളയും തമ്മില്‍ ഇലക്ട്രോണിക്‌സ്, ..."
1,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഒന്‍പതാമത് വൈബ്രന്റ് ഗുജറാത്ത് ഉച്ചകോടിക്ക് ഗാ...,വൈബ്രന്റ് ഗുജറാത്ത് ഉച്ചകോടിക്ക് മുന്നോടിയായുള...
2,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ക്ഷയരോഗ നിര്‍മ്മാര്‍ജ്ജനത്തിനായുള്ള ഡല്‍ഹി ഉച്...,ക്ഷയരോഗ നിര്‍മ്മാര്‍ജ്ജന ഉച്ചകോടി പ്രധാനമന്ത്ര...
3,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഭക്ഷ്യസുരക്ഷയിലും അനുബന്ധ മേഖലകളിലും സഹകരിക്കു...,ഭക്ഷ്യസുരക്ഷയിലും അനുബന്ധ മേഖലകളിലും സഹകരിക്കു...
4,https://www.pmindia.gov.in/ml/news_updates/%E0...,https://www.pmindia.gov.in/ml/news_updates/%E0...,ഉത്തരാഖണ്ഡിലെ പൗരിയിൽ ബസ് അപകടത്തിലെ ജീവഹാനിയി...,ഉത്തരാഖണ്ഡിലെ പൗരി ബസ്സപകടത്തിൽ പ്രധാനമന്ത്രി...


In [ ]:
df = df.rename(columns={'Text': 'text', 'Summary': 'summary'})

In [ ]:
from datasets import Dataset, load_metric

dataset = Dataset.from_pandas(df[['text', 'summary']])

In [ ]:
texts = df['text'].tolist()
references = df['summary'].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai4bharat/IndicBART"  # Malayalam IndicBART

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")


# **IndicBART**

In [ ]:
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator


def malayalam_to_devanagari(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'ml', 'hi') + ' </s> <2hi>'
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'ml', 'hi') + ' </s> <2hi>' for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

def devanagari_to_malayalam(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'hi', 'ml')
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'hi', 'ml') for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

In [ ]:
texts = texts[:16]

In [ ]:
batch_size = 8  # smaller batch size to show progress in chunks
predictions = []

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]
    batch_texts = malayalam_to_devanagari(batch_texts)
    # inputs = tokenizer(batch_texts, max_length=512, truncation=True, padding=True, return_tensors='pt')
    # inputs = tokenizer("मैं [MASK] हूँ </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids
    inputs = tokenizer(batch_texts, add_special_tokens=False, padding=True, return_tensors='pt')

    # summary_ids = model.generate(
    #     inputs['input_ids'], attention_mask=inputs['attention_mask'],
    #     max_length=128, num_beams=4, early_stopping=True
    # )

    summary_ids = model.generate(
        inputs['input_ids'], use_cache=True, num_beams=4, max_length=128,
        early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id,
        eos_token_id=eos_id,
        decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>")
        )

    batch_preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    predictions.extend(batch_preds)

predictions = devanagari_to_malayalam(predictions)

Generating summaries:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
input_text = """
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.

Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.

TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.
"""

# Generate summary in English
summary_output = model.generate(
    tokenizer(input_text, return_tensors="pt", padding=True, add_special_tokens=False).input_ids,
    use_cache=True, num_beams=4, max_length=128,
    early_stopping=False, pad_token_id=pad_id, bos_token_id=bos_id,
    eos_token_id=eos_id,
    decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>")
)

# Decode and print summary
summary_text = tokenizer.decode(summary_output[0], skip_special_tokens=True)
print(summary_text)


Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a


In [ ]:
# Load metrics
rouge = load_metric("rouge")
bleu = load_metric("bleu")

# Compute ROUGE scores
rouge_results = rouge.compute(predictions=predictions, references=references[:16], use_stemmer=True)
print("ROUGE-1 F1 Score:", round(rouge_results['rouge1'].mid.fmeasure, 4))
print("ROUGE-2 F1 Score:", round(rouge_results['rouge2'].mid.fmeasure, 4))
print("ROUGE-L F1 Score:", round(rouge_results['rougeL'].mid.fmeasure, 4))

# Prepare references for BLEU (tokenized, list of lists of tokens)
refs_for_bleu = [[ref.split()] for ref in references[:16]]
preds_for_bleu = [pred.split() for pred in predictions]

# Compute BLEU
bleu_results = bleu.compute(predictions=preds_for_bleu, references=refs_for_bleu)
print("BLEU Score:", round(bleu_results['bleu'], 4))

# Show 5 sample predictions with inputs and references
print("\nSample Predictions:\n")
for i in range(5):
    print(f"Input Text:\n{texts[i]}\n")
    print(f"Generated Summary:\n{predictions[i]}\n")
    print(f"Reference Summary:\n{references[i]}\n")
    print("="*80)

ROUGE-1 F1 Score: 0.05
ROUGE-2 F1 Score: 0.0417
ROUGE-L F1 Score: 0.05
BLEU Score: 0.0311

Sample Predictions:

Input Text:
ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സിക

In [ ]:
df.count()

,0
source_url,116
target_url,116
text,116
summary,116


# Qwen

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_name = "Qwen/Qwen1.5-0.5B"  # or Qwen1.5-1.8B for stronger

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).cuda()


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
predictions=[]
torch.cuda.empty_cache()

In [ ]:
for text in tqdm(texts, desc="Generating summaries"):
    try:
        prompt = f"ഒരു ചെറിയ സാരാംശം തരുക:\n{text.strip()}\nസാരാംശം:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,
                num_beams=2,  # reduce if OOM
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id  # required for Qwen
            )

        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        if "സാരാംശം:" in generated_summary:
          summary_clean = generated_summary.split("സാരാംശം:")[-1].strip()
        else:
          summary_clean = generated_summary.strip()

        predictions.append(summary_clean)

        torch.cuda.empty_cache()

    except torch.cuda.OutOfMemoryError:
        print("⚠️ CUDA OOM – Skipping this input.")
        torch.cuda.empty_cache()
        predictions.append("[OOM ERROR – Skipped]")


Generating summaries:   0%|          | 0/116 [00:00<?, ?it/s]

⚠️ CUDA OOM – Skipping this input.


Generating summaries:   1%|          | 1/116 [00:00<00:57,  1.99it/s]

⚠️ CUDA OOM – Skipping this input.


Generating summaries:   5%|▌         | 6/116 [00:13<03:51,  2.10s/it]

...

Generating summaries:  96%|█████████▌| 111/116 [04:13<00:12,  2.43s/it]

⚠️ CUDA OOM – Skipping this input.


Generating summaries:  97%|█████████▋| 113/116 [04:17<00:06,  2.28s/it]

⚠️ CUDA OOM – Skipping this input.


Generating summaries: 100%|██████████| 116/116 [04:26<00:00,  2.29s/it]


In [ ]:
!pip install rouge_score
!pip install sacrebleu

In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")
bleu = load_metric("bleu")

# ROUGE
rouge_result = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
print("ROUGE-1:", round(rouge_result['rouge1'].mid.fmeasure, 4))
print("ROUGE-2:", round(rouge_result['rouge2'].mid.fmeasure, 4))
print("ROUGE-L:", round(rouge_result['rougeL'].mid.fmeasure, 4))

# BLEU
bleu_result = bleu.compute(
    predictions=[p.split() for p in predictions],
    references=[[r.split()] for r in references]
)
print("BLEU:", round(bleu_result["bleu"], 4))


ROUGE-1: 0.0172
ROUGE-2: 0.0086
ROUGE-L: 0.0172
BLEU: 0.0676


In [ ]:
for i in range(5):
    print(f"\nInput: {texts[i]}")
    print(f"Generated: {predictions[i]}")
    print(f"Reference: {references[i]}")
    print("="*70)



Input: ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍ ഇന്ത്യ തുടങ്ങിയ

# Gemma



In [ ]:
from huggingface_hub import login
login()  #

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2b-it"  # or "gemma-7b-it" if you have enough memory
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
import torch

predictions = []
torch.cuda.empty_cache()

for text in tqdm(texts, desc="Generating summaries"):
    try:
        prompt = f"<start_of_turn>user\nഒരു ചെറിയ സാരാംശം തരുക:\n{text.strip()}<end_of_turn>\n<start_of_turn>model\n"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,
                num_beams=1,
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        # Optional cleanup: if the model repeats the prompt, trim it
        if "<start_of_turn>model" in generated_summary:
            summary_clean = generated_summary.split("<start_of_turn>model")[-1].strip()
        else:
            summary_clean = generated_summary

        predictions.append(summary_clean)

        torch.cuda.empty_cache()

    except torch.cuda.OutOfMemoryError:
        print("⚠️ CUDA OOM – Skipping this input.")
        torch.cuda.empty_cache()
        predictions.append("[OOM ERROR – Skipped]")


Generating summaries:  13%|█▎        | 15/116 [1:07:46<7:32:26, 268.78s/it]The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
from evaluate import load
rouge = load("rouge")
bleu = load("bleu")  # optional

# Filter out OOM or mismatched examples
filtered_predictions = [p for p in predictions if not p.startswith("[OOM")]
filtered_references = [r for p, r in zip(predictions, references) if not p.startswith("[OOM")]

# Compute ROUGE
results = rouge.compute(predictions=filtered_predictions, references=filtered_references, use_stemmer=True)
print("ROUGE-1:", results["rouge1"].mid.fmeasure)


## By-T5-Small

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ByT5 and Flan-T5
byt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small").to(device)
byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

In [ ]:
def generate_summary(text, model, tokenizer, prefix=""):
    prompt = prefix + text.strip()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=100,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from tqdm.auto import tqdm
import torch

In [ ]:
byt5_preds = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="ByT5-Small"):
    text, reference = row['text'], row['summary']
    generated_summary = generate_summary(text, byt5_model, byt5_tokenizer)
    byt5_preds.append(generated_summary)

# Save outputs
df['byt5_summary'] = byt5_preds

ByT5-Small:   0%|          | 0/116 [00:00<?, ?it/s]

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_scores(references, predictions):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth_fn = SmoothingFunction().method1

    rouge1_list, rouge2_list, rougeL_list, bleu_list = [], [], [], []

    for ref, pred in zip(references, predictions):
        ref = ref.strip()
        pred = pred.strip()

        scores = rouge.score(ref, pred)
        rouge1_list.append(scores['rouge1'].fmeasure)
        rouge2_list.append(scores['rouge2'].fmeasure)
        rougeL_list.append(scores['rougeL'].fmeasure)

        ref_tokens = [nltk.word_tokenize(ref)]
        pred_tokens = nltk.word_tokenize(pred)
        bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth_fn)
        bleu_list.append(bleu)

    avg_rouge1 = sum(rouge1_list) / len(rouge1_list)
    avg_rouge2 = sum(rouge2_list) / len(rouge2_list)
    avg_rougeL = sum(rougeL_list) / len(rougeL_list)
    avg_bleu = sum(bleu_list) / len(bleu_list)

    return {
        "ROUGE-1": avg_rouge1,
        "ROUGE-2": avg_rouge2,
        "ROUGE-L": avg_rougeL,
        "BLEU": avg_bleu
    }


In [ ]:
# For ByT5-small
byt5_scores = evaluate_scores(df['summary'], df['byt5_summary'])
print("ByT5-small scores:")
for metric, score in byt5_scores.items():
    print(f"{metric}: {score:.4f}")

ByT5-small scores:
ROUGE-1: 0.0149
ROUGE-2: 0.0043
ROUGE-L: 0.0149
BLEU: 0.0241


In [ ]:
for i in range(5):
    print(f"\nInput: {texts[i]}")
    print(f"Generated: {byt5_preds[i]}")
    print(f"Reference: {references[i]}")
    print("="*70)



Input: ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍ ഇന്ത്യ തുടങ്ങിയ

# Flan

In [ ]:
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(device)
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def generate_summary(text):
    # Prepare prompt (instruction + input text)
    prompt = f"Summarize the following Malayalam text in malayalam:\n{text}"

    # Tokenize input text
    inputs = flan_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)

    # Generate summary
    summary_ids = flan_model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=4,
        early_stopping=True,
        do_sample=False,
    )
    # Decode the output summary
    summary = flan_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
flan_preds = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Flan-T5-Small"):
    text, reference = row['text'], row['summary']
    summary = generate_summary(text)
    flan_preds.append(summary)

# Save outputs
df['flan_summary'] = flan_preds

Flan-T5-Small:   0%|          | 0/116 [00:00<?, ?it/s]

In [ ]:

# For Flan-T5-small
flan_scores = evaluate_scores(df['summary'], df['flan_summary'])
print("\nFlan-T5-small scores:")
for metric, score in flan_scores.items():
    print(f"{metric}: {score:.4f}")



Flan-T5-small scores:
ROUGE-1: 0.0000
ROUGE-2: 0.0000
ROUGE-L: 0.0000
BLEU: 0.0000


In [ ]:
for i in range(5):
    print(f"\nInput: {texts[i]}")
    print(f"Generated: {flan_preds[i]}")
    print(f"Reference: {references[i]}")
    print("="*70)


Input: ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍ ഇന്ത്യ തുടങ്ങിയ

# **mT5**


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
input_text = "summarize: കേരളത്തിൽ ഇന്നത്തെ കാലാവസ്ഥയിൽ പെയ്യുന്ന കനത്ത മഴ യാത്രക്കാർക്ക് വലിയ തടസ്സം സൃഷ്ടിച്ചു."

# Tokenize input
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)


In [ ]:
summary_ids = model.generate(
    input_ids,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

# Decode the output
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
print("Summary:", summary)


Summary: ['<extra_id_0>.']


In [ ]:
def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
predictions = []
for text in tqdm(texts[:5]):
    pred = generate_summary(text)
    predictions.append(pred)


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for i in range(5):
    print(f"\nExample {i+1}")
    print("="*40)
    print("Input Text:\n", texts[i])
    print("\nReference Summary:\n", references[i])
    print("\nPredicted Summary:\n", predictions[i])
    print("="*40)


Example 1
Input Text:
 ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍

In [1]:
!apt-get install git -y

# Step 2: Configure Git with your GitHub email and username
!git config --global user.email "anushreevinod10.com"
!git config --global user.name "anushree-vinod"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
!git clone https://github.com/anushree-vinod/LLM_Evaluation_Malayalam.git

Cloning into 'LLM_Evaluation_Malayalam'...


In [5]:
%cd /content/LLM_Evaluation_Malayalam

/content/LLM_Evaluation_Malayalam


In [6]:
!git add .

!git commit -m "Initial commit from Google Colab"


On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
